In [1]:
import cv2
import numpy as np

In [13]:
def pre_process(path):
    image = cv2.imread(path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    scale_factor = 700 / max(image.shape[:2])
    image = cv2.resize(image, (0, 0), fx=scale_factor, fy=scale_factor)
    gray = cv2.resize(gray, (0, 0), fx=scale_factor, fy=scale_factor)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV, 11, 2)
    return (image, thresh, scale_factor)
    
def show(img,time = 3):
    cv2.imshow("Image", img)
    cv2.waitKey(int(time * 1000))
    cv2.destroyAllWindows()

def edge(image, thresh, scale_factor):
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    coin_contours = []
    for cnt in contours:
        perimeter = cv2.arcLength(cnt, True)
        area = cv2.contourArea(cnt)
        if perimeter:
            circularity = 4 * np.pi * (area / (perimeter ** 2))
            if 0.7 < circularity < 1.2 and area > 500 * (scale_factor ** 2):  
                coin_contours.append(cnt)
    cv2.drawContours(image, coin_contours, -1, (0, 255, 0), 2)
    show(image,3)
    return coin_contours

    
def contour_segmentation(image, coin_contours):
    segmented_coins = []
    for cnt in coin_contours:
        (x, y), radius = cv2.minEnclosingCircle(cnt)
        center = (int(x), int(y))
        radius = int(radius)
        mask = np.zeros_like(image, dtype=np.uint8)
        cv2.circle(mask, center, radius, (255, 255, 255), -1)
        coin_segment = cv2.bitwise_and(image, mask)
        x1, y1, x2, y2 = center[0] - radius, center[1] - radius, center[0] + radius, center[1] + radius
        coin_segment = coin_segment[y1:y2, x1:x2]
        segmented_coins.append(coin_segment)
    for i, coin in enumerate(segmented_coins):
        show(coin,2)
    return segmented_coins

    
def count_coin(coin_contours, segmented_coins):
    return (len(coin_contours), len(segmented_coins))

In [14]:
# import cv2
# import numpy as np

# def segment_coins_knn(path):
#     # Load the image
#     image = cv2.imread(path)
#     image_copy = image.copy()

#     # SCALE-INVARIANT RESIZING
#     scale_factor = 700 / max(image.shape[:2])  
#     image = cv2.resize(image, (0, 0), fx=scale_factor, fy=scale_factor)

#     # Convert image to LAB color space and extract L-channel
#     lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
#     L_channel = lab[:, :, 0]  # Extract only the Luminance channel

#     # Get image dimensions
#     h, w = L_channel.shape

#     # Generate (x, y) coordinate grid
#     x_coords, y_coords = np.meshgrid(np.arange(w), np.arange(h))

#     # Flatten all data
#     pixel_values = np.column_stack((x_coords.flatten(), y_coords.flatten(), L_channel.flatten()))
#     pixel_values = np.float32(pixel_values)

#     # Apply K-Means Clustering
#     k = 8  # Number of clusters (background + coins)
#     criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.2)
#     _, labels, centers = cv2.kmeans(pixel_values, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

#     # Convert labels to mask (selecting the cluster with lowest brightness)
#     labels = labels.reshape((h, w))
#     mask = np.uint8(labels == np.argmin(centers[:, 2])) * 255  # Coins are usually darker

#     # Apply Otsu’s thresholding for cleaner segmentation
#     _, mask = cv2.threshold(mask, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

#     # Morphological operations to refine mask
#     kernel = np.ones((5, 5), np.uint8)
#     mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=2)  # Fill small holes
#     mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=1)  # Remove noise

#     show(mask)
#     # Find contours of the segmented coins
#     # contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#     # coin_contours = []
#     # for cnt in contours:
#     #     perimeter = cv2.arcLength(cnt, True)
#     #     area = cv2.contourArea(cnt)
#     #     if perimeter:
#     #         circularity = 4 * np.pi * (area / (perimeter ** 2))
#     #         # if 0.7 < circularity < 1.2 and area > 2000 * (scale_factor ** 2):  
#     #         coin_contours.append(cnt)
#     # cv2.drawContours(image, coin_contours, -1, (0, 255, 0), 2)

#     # # Show segmented result
#     # show(image)

#     # return coin_contours  # Return segmented coin images

# # Run function on image
# path = "coins3.jpg"
# coins = segment_coins_knn(path)

In [15]:
def main(path):
    image, thresh,scale_factor = pre_process(path)
    coin_contours = edge(image, thresh, scale_factor)
    segmented_coins = contour_segmentation(image, coin_contours)
    print(count_coin(coin_contours, segmented_coins))

In [16]:
# main("coins.jpg")
# main("coins1.jpg")
# main("coins2.jpg")
# main("coins3.jpg")
# main("coins4.jpg")
# main("coins5.jpg")
# main("coins6.jpg")
# main("coins7.jpg")
# # main("coins8.jpg")
# main("coins9.jpg")
# main("coins10.jpg")
# main("coins11.jpg")
# main("coins12.jpg")
# main("coins13.jpg")
main("coins14.jpg")
# main("coins15.png")

(4, 4)


In [ ]:
# segment_coins_watershed("coins.jpg")
# segment_coins_watershed("coins1.jpg")
# segment_coins_watershed("coins2.jpg")
segment_coins_knn("coins3.jpg")
# segment_coins_watershed("coins4.jpg")
# segment_coins_watershed("coins5.jpg")
# segment_coins_watershed("coins6.jpg")
# segment_coins_watershed("coins7.jpg")
# # segment_coins_watershed("coins8.jpg")
# segment_coins_watershed("coins9.jpg")
# segment_coins_watershed("coins10.jpg")


In [5]:
import cv2
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans

def pre_process(path):
    # Read and preprocess the image
    image = cv2.imread(path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Resize the image to ensure scale invariance
    scale_factor = 700 / max(image.shape[:2])
    image = cv2.resize(image, (0, 0), fx=scale_factor, fy=scale_factor)
    gray = cv2.resize(gray, (0, 0), fx=scale_factor, fy=scale_factor)
    
    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    
    return image, blurred, scale_factor

def show(img, time=3):
    # Display the image for a specified time
    cv2.imshow("Image", img)
    cv2.waitKey(int(time * 1000))
    cv2.destroyAllWindows()

def knn_segmentation(image, blurred, scale_factor):
    # Reshape the image into a 2D array of pixels (rows = pixels, columns = color channels + spatial coordinates)
    h, w, _ = image.shape
    pixels = image.reshape((-1, 3))  # Color features (BGR)
    y_coords, x_coords = np.mgrid[0:h, 0:w]
    coords = np.vstack((x_coords.ravel(), y_coords.ravel())).T  # Spatial features (x, y)
    
    # Combine color and spatial features
    features = np.hstack((pixels, coords))
    
    # Apply K-Means clustering to find dominant colors (used as labels for KNN)
    kmeans = KMeans(n_clusters=5)  # Adjust the number of clusters based on the number of coins
    kmeans.fit(pixels)
    labels = kmeans.labels_
    
    # Train a KNN classifier on the combined features and labels
    knn = KNeighborsClassifier(n_neighbors=5)
    knn.fit(features, labels)
    
    # Predict labels for all pixels
    segmented_labels = knn.predict(features)
    
    # Reshape the labels back into the original image shape
    segmented_image = segmented_labels.reshape((h, w))
    
    # Convert the segmented image to CV_8U for display
    segmented_image_display = np.uint8(255 * (segmented_image / np.max(segmented_image)))
    
    # Create a mask for each coin (cluster)
    segmented_coins = []
    for label in np.unique(segmented_labels):
        mask = np.zeros_like(blurred, dtype=np.uint8)  # Use blurred image as reference for shape
        mask[segmented_image == label] = 255
        
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    coin_contours = []
    for cnt in contours:
        perimeter = cv2.arcLength(cnt, True)
        area = cv2.contourArea(cnt)
        if perimeter:
            circularity = 4 * np.pi * (area / (perimeter ** 2))
            if 0.7 < circularity < 1.2 and area > 2000 * (scale_factor ** 2):  
                coin_contours.append(cnt)

    print(coin_contours)
    for cnt in coin_contours:
        
        (x, y), radius = cv2.minEnclosingCircle(cnt)
        center = (int(x), int(y))
        radius = int(radius)
        mask = np.zeros_like(image, dtype=np.uint8)
        cv2.circle(mask, center, radius, (255, 255, 255), -1)
        coin_segment = cv2.bitwise_and(image, mask)
        x1, y1, x2, y2 = center[0] - radius, center[1] - radius, center[0] + radius, center[1] + radius
        coin_segment = coin_segment[y1:y2, x1:x2]
        segmented_coins.append(coin_segment)
    
    return segmented_coins, segmented_image_display

# Main execution
if __name__ == "__main__":
    # Load and preprocess the image
    path = "coins14.jpg"  # Replace with your image path
    image, blurred, scale_factor = pre_process(path)
    
    # Perform KNN-based segmentation
    segmented_coins, segmented_image = knn_segmentation(image, blurred, scale_factor)
    
    # Display the segmented image
    show(segmented_image, 5)
    
    # Display each segmented coin
    for i, coin in enumerate(segmented_coins):
        show(coin, 2)

KeyboardInterrupt: 

In [11]:
print(segmented_image.astype(np.int8))

[[2 2 0 ... 3 3 3]
 [2 2 0 ... 3 3 3]
 [2 2 2 ... 3 3 3]
 ...
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]]


In [ ]:
segmented_coins

NameError: name 'coin_contours' is not defined

In [2]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from copy import deepcopy

def gaussian_smoothing(input_img):
                                
    gaussian_filter=np.array([[0.109,0.111,0.109],
                              [0.111,0.135,0.111],
                              [0.109,0.111,0.109]])
                                
    return cv2.filter2D(input_img,-1,gaussian_filter)  
        
def canny_edge_detection(input):
    
    input = input.astype('uint8')

    # Using OTSU thresholding - bimodal image
    otsu_threshold_val, ret_matrix = cv2.threshold(input,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    
    #lower_threshold = otsu_threshold_val * 0.8
    #upper_threshold = otsu_threshold_val * 1.7
    
    lower_threshold = otsu_threshold_val * 0.4
    upper_threshold = otsu_threshold_val * 1.3
    
    print(lower_threshold,upper_threshold)
    
    #print(lower_threshold,upper_threshold)
    edges = cv2.Canny(input, lower_threshold, upper_threshold)
    return edges
    
def HoughCircles(input,circles): 
    rows = input.shape[0] 
    cols = input.shape[1] 
    
    # initializing the angles to be computed 
    sinang = dict() 
    cosang = dict() 
    
    # initializing the angles  
    for angle in range(0,360): 
        sinang[angle] = np.sin(angle * np.pi/180) 
        cosang[angle] = np.cos(angle * np.pi/180) 
            
    # initializing the different radius
    # For Test Image <----------------------------PLEASE SEE BEFORE RUNNING------------------------------->
    #radius = [i for i in range(10,70)]
    #For Generic Images
    length=int(rows/2)
    radius = [i for i in range(5,length)]

       
    # Initial threshold value 
    threshold = 190 
    
    for r in radius:
        #Initializing an empty 2D array with zeroes 
        acc_cells = np.full((rows,cols),fill_value=0,dtype=np.uint64)
         
        # Iterating through the original image 
        for x in range(rows): 
            for y in range(cols): 
                if input[x][y] == 255:# edge 
                    # increment in the accumulator cells 
                    for angle in range(0,360): 
                        b = y - round(r * sinang[angle]) 
                        a = x - round(r * cosang[angle]) 
                        if a >= 0 and a < rows and b >= 0 and b < cols: 
                            acc_cells[a][b] += 1
                             
        print('For radius: ',r)
        acc_cell_max = np.amax(acc_cells)
        print('max acc value: ',acc_cell_max)
        
        if(acc_cell_max > 150):  

            print("Detecting the circles for radius: ",r)       
            
            # Initial threshold
            acc_cells[acc_cells < 150] = 0  
               
            # find the circles for this radius 
            for i in range(rows): 
                for j in range(cols): 
                    if(i > 0 and j > 0 and i < rows-1 and j < cols-1 and acc_cells[i][j] >= 150):
                        avg_sum = np.float32((acc_cells[i][j]+acc_cells[i-1][j]+acc_cells[i+1][j]+acc_cells[i][j-1]+acc_cells[i][j+1]+acc_cells[i-1][j-1]+acc_cells[i-1][j+1]+acc_cells[i+1][j-1]+acc_cells[i+1][j+1])/9) 
                        print("Intermediate avg_sum: ",avg_sum)
                        if(avg_sum >= 33):
                            print("For radius: ",r,"average: ",avg_sum,"\n")
                            circles.append((i,j,r))
                            acc_cells[i:i+5,j:j+7] = 0
                 

def main():
    
    img_path = 'coins3.jpg'
    #sample_inp1_path = 'circle_sample_1.jpg'
    
    orig_img = cv2.imread(img_path)
    
    # Reading the input image and converting to gray scale
    input = cv2.imread(img_path,cv2.IMREAD_GRAYSCALE)
    
    # Create copy of the orignial image
    input_img = deepcopy(input)
    
    #Steps
    #1. Denoise using Gaussian filter and detect edges using canny edge detector
    smoothed_img = gaussian_smoothing(input_img)
    
    #2. Detect Edges using Canny Edge Detector
    edged_image = canny_edge_detection(smoothed_img)
    #3. Detect Circle radius
    #4. Perform Circle Hough Transform
    circles = []
    
    # cv2.imshow('Circle Detected Image',edged_image)
   
    # Detect Circle 
    HoughCircles(edged_image,circles)  
    
    # Print the output
    for vertex in circles:
        cv2.circle(orig_img,(vertex[1],vertex[0]),vertex[2],(0,255,0),1)
        cv2.rectangle(orig_img,(vertex[1]-2,vertex[0]-2),(vertex[1]-2,vertex[0]-2),(0,0,255),3)
        
    print(circles)
        
    cv2.imshow('Circle Detected Image',orig_img) 
    cv2.imwrite('Circle_Detected_Image.jpg',orig_img) 

# if __name__ == '__main__':
main()


50.800000000000004 165.1
For radius:  5
max acc value:  242
Detecting the circles for radius:  5
Intermediate avg_sum:  53.555557
For radius:  5 average:  53.555557 

Intermediate avg_sum:  17.333334
Intermediate avg_sum:  16.88889
Intermediate avg_sum:  16.88889
Intermediate avg_sum:  17.11111
Intermediate avg_sum:  75.888885
For radius:  5 average:  75.888885 

Intermediate avg_sum:  18.0
Intermediate avg_sum:  16.666666
Intermediate avg_sum:  16.666666
Intermediate avg_sum:  17.333334
Intermediate avg_sum:  16.666666
Intermediate avg_sum:  19.0
Intermediate avg_sum:  18.666666
Intermediate avg_sum:  38.22222
For radius:  5 average:  38.22222 

Intermediate avg_sum:  57.77778
For radius:  5 average:  57.77778 

Intermediate avg_sum:  17.0
Intermediate avg_sum:  16.666666
Intermediate avg_sum:  19.333334
Intermediate avg_sum:  16.88889
Intermediate avg_sum:  18.88889
Intermediate avg_sum:  17.0
Intermediate avg_sum:  17.777779
Intermediate avg_sum:  38.0
For radius:  5 average:  38.0 

KeyboardInterrupt: 

In [1]:
import numpy as np
import imutils
import cv2 as cv

class Stitcher:
	def __init__(self):
		# determine if we are using OpenCV v3.X
		self.isv3 = imutils.is_cv3(or_better = True)

	def stitch(self, images, ratio = 0.75, reprojThresh = 4.0, showMatches = False):
		# unpack the images, then detect interestpoints and extract local invariant descriptors from them
		(imageB, imageA) = images
		(interestPointsA, featuresA) = self.detectAndDescribe(imageA)
		(interestPointsB, featuresB) = self.detectAndDescribe(imageB)
		# match features between the two images
		M = self.matchInterestPoints(interestPointsA, interestPointsB, featuresA, featuresB, ratio, reprojThresh)
		# if the match is None, then there aren't enough matched interestpoints to create a panorama
		if M is None:
			return None
		# otherwise, apply a perspective warp to stitch the images together
		(matches, H, status) = M
		stitchedImage = cv.warpPerspective(imageA, H, (imageA.shape[1] + imageB.shape[1], imageA.shape[0]))
		stitchedImage[0:imageB.shape[0], 0:imageB.shape[1]] = imageB
		# check to see if the interestpoint matches should be visualized
		if showMatches:
			visualization = self.drawMatches(imageA, imageB, interestPointsA, interestPointsB, matches, status)
			# return a tuple of the stitched image and the visualization
			return (stitchedImage, visualization)
		# return the stitched image
		return stitchedImage
	
	def detectAndDescribe(self, image):
		# convert the image to grayscale
		gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
		# check to see if we are using OpenCV 3.X
		if self.isv3:
			# detect and extract features from the image
			descriptor = cv.xfeatures2d.SIFT_create()
			(kps, features) = descriptor.detectAndCompute(image, None)
		# otherwise, we are using OpenCV 2.4.X
		else:
			# detect interestpoints in the image
			detector = cv.FeatureDetector_create("SIFT")
			kps = detector.detect(gray)
			# extract features from the image
			extractor = cv.DescriptorExtractor_create("SIFT")
			(kps, features) = extractor.compute(gray, kps)
		# convert the interestpoints from interestPoint objects to NumPy arrays
		kps = np.float32([kp.pt for kp in kps])
		# return a tuple of interestpoints and features
		return (kps, features)

	def matchInterestPoints(self, interestPointsA, interestPointsB, featuresA, featuresB, ratio, reprojThresh):
		# compute the raw matches and initialize the list of actual matches
		matcher = cv.DescriptorMatcher_create("BruteForce")
		rawMatches = matcher.knnMatch(featuresA, featuresB, 2)
		matches = []
		# loop over the raw matches
		for m in rawMatches:
			# ensure the distance is within a certain ratio of each other (i.e. Lowe's ratio test)
			if len(m) == 2 and m[0].distance < m[1].distance * ratio:
				matches.append((m[0].trainIdx, m[0].queryIdx))
		# computing a homography requires at least 4 matches
		if len(matches) > 4:
			# construct the two sets of points
			ptsA = np.float32([interestPointsA[i] for (_, i) in matches])
			ptsB = np.float32([interestPointsB[i] for (i, _) in matches])
			# compute the homography between the two sets of points
			(H, status) = cv.findHomography(ptsA, ptsB, cv.RANSAC,
				reprojThresh)
			# return the matches along with the homograpy matrix and status of each matched point
			return (matches, H, status)
		# otherwise, no homograpy could be computed
		return None

	def drawMatches(self, imageA, imageB, interestPointsA, interestPointsB, matches, status):
		# initialize the output visualization image
		(hA, wA), (hB, wB) = imageA.shape[:2], imageB.shape[:2]
		visualization = np.zeros((max(hA, hB), wA + wB, 3), dtype = "uint8")
		visualization[0:hA, 0:wA], visualization[0:hB, wA:] = imageA, imageB
		# loop over the matches
		for ((trainIdx, queryIdx), s) in zip(matches, status):
			# only process the match if the interest point was successfully matched
			if s == 1:
				# draw the match
				ptA = (int(interestPointsA[queryIdx][0]), int(interestPointsA[queryIdx][1]))
				ptB = (int(interestPointsB[trainIdx][0]) + wA, int(interestPointsB[trainIdx][1]))
				cv.line(visualization, ptA, ptB, (70, 90, 255), 1)
		# return the visualization
		return visualization

In [ ]:

import imutils
import cv2 as cv

imageA = cv.imread("left2.jpg")
imageB = cv.imread("right2.jpg")
imageA = imutils.resize(imageA, width = 600)
imageB = imutils.resize(imageB, width = 600)

stitcher = Stitcher() # Create an object of the Stitcher class to stitch the images together to create a panorama
(stichedImage, visualization) = stitcher.stitch([imageA, imageB], showMatches = True)

cv.imshow("Image A", imageA)
cv.imshow("Image B", imageB)
cv.imshow("Matched Interest Points", visualization)
cv.imwrite("InterestPoints2.jpg", visualization)
cv.imshow("Stitched Image", stichedImage)
cv.imwrite("stitched2.jpg", stichedImage)
cv.waitKey(0)

[ WARN:0@3.818] global shadow_sift.hpp:13 SIFT_create DEPRECATED: cv.xfeatures2d.SIFT_create() is deprecated due SIFT tranfer to the main repository. https://github.com/opencv/opencv/issues/16736


27

: 